In [1]:
!pip install keras_cv_attention_models -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.83.0 which is incompatible.
tfx-bsl 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
tensorflow-transform 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.44.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras_cv_attention_models import convnext

In [3]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/train/',
        target_size=(128, 128),
        batch_size=8,
        class_mode='binary')

validation_generator = train_datagen.flow_from_directory(
        '/kaggle/input/chest-xray-pneumonia/chest_xray/test/',
        target_size=(128, 128),
        batch_size=8,
        class_mode='binary')

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [5]:
convnext_base = convnext.ConvNeXtBase(input_shape=(128, 128, 3), pretrained=None, num_classes=0)
x = tf.keras.layers.GlobalAveragePooling2D()(convnext_base.output)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)
convnext_base_model = tf.keras.Model(convnext_base.input, x)

In [6]:
METRICS = [
        'accuracy',
    ]
    
convnext_base_model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=METRICS
    )

In [7]:
def exponential_decay(lr0, s):
    def exponential_decay_fn(epoch):
        return lr0 * 0.1 **(epoch / s)
    return exponential_decay_fn

exponential_decay_fn = exponential_decay(0.01, 20)

lr_scheduler = tf.keras.callbacks.LearningRateScheduler(exponential_decay_fn)

In [8]:
history = convnext_base_model.fit(
    train_generator,
    epochs=10,
    validation_data=validation_generator,
    callbacks=[lr_scheduler]
)

Epoch 1/10


2023-04-18 19:28:32.615727: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel/stack1_block2_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


652/652 [==============================] - 224s 233ms/step - loss: 4.1606 - accuracy: 0.7078 - val_loss: 91.4135 - val_accuracy: 0.3750 - lr: 0.0100
Epoch 2/10
652/652 [==============================] - 130s 200ms/step - loss: 36.4077 - accuracy: 0.6215 - val_loss: 1.3506 - val_accuracy: 0.6250 - lr: 0.0089
Epoch 3/10
652/652 [==============================] - 131s 200ms/step - loss: 2.0274 - accuracy: 0.6265 - val_loss: 0.7110 - val_accuracy: 0.3750 - lr: 0.0079
Epoch 4/10
652/652 [==============================] - 131s 201ms/step - loss: 0.8523 - accuracy: 0.6446 - val_loss: 0.6681 - val_accuracy: 0.6250 - lr: 0.0071
Epoch 5/10
652/652 [==============================] - 131s 201ms/step - loss: 0.6840 - accuracy: 0.6869 - val_loss: 0.6622 - val_accuracy: 0.6250 - lr: 0.0063
Epoch 6/10
652/652 [==============================] - 132s 202ms/step - loss: 0.6419 - accuracy: 0.7063 - val_loss: 0.7963 - val_accuracy: 0.6250 - lr: 0.0056
Epoch 7/10
652/652 [==============================] - 1

In [9]:
convnext_base_model.save("CovnNeXT_xray_Pneumonia.h5")